<a href="https://colab.research.google.com/github/ksharat45/Pytorch/blob/main/pytorch_tensor_internals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

# 1. Original Contiguous Tensor
original_tensor = torch.arange(10)
print(f"Original Tensor: {original_tensor}")
print(f"Original Data Pointer: {original_tensor.data_ptr()}")
print("---")

## Case A: Using view() on a Contiguous Tensor
print("## Case A: view() (Always creates a View if Contiguous) ##")
view_tensor = original_tensor.view(2, 5)
print(f"View Tensor: {view_tensor}")
print(f"View Data Pointer: {view_tensor.data_ptr()}")

# Check for shared memory
is_view_shared = original_tensor.data_ptr() == view_tensor.data_ptr()
print(f"Memory Shared (view)? {is_view_shared} (It's a VIEW)")
print("--------------------")

## Case B: Using reshape() on a Contiguous Tensor
print("## Case B: reshape() (Creates a View when Contiguous) ##")
reshape_view_tensor = original_tensor.reshape(5, 2)
print(f"Reshape (View) Tensor: {reshape_view_tensor}")
print(f"Reshape (View) Data Pointer: {reshape_view_tensor.data_ptr()}")

# Check for shared memory
is_reshape_view_shared = original_tensor.data_ptr() == reshape_view_tensor.data_ptr()
print(f"Memory Shared (reshape/view)? {is_reshape_view_shared} (It's a VIEW)")
print("--------------------")

## Case C: Forcing reshape() to create a Copy
print("## Case C: reshape() (Creates a Copy when Non-Contiguous) ##")
# 1. Create a NON-CONTIGUOUS tensor using transpose()
non_contiguous_tensor = torch.arange(10).reshape(2, 5).T
print(f"non_contiguous_tensor(view): {non_contiguous_tensor}")
print(f"Non-Contiguous Data Pointer: {non_contiguous_tensor.data_ptr()}")
# Note: Calling .view() on this tensor would raise an error!

# 2. Call reshape() on the non-contiguous tensor
reshape_copy_tensor = non_contiguous_tensor.reshape(10)
print(f"Reshape (Copy) Tensor: {reshape_copy_tensor}")
print(f"Reshape (Copy) Data Pointer: {reshape_copy_tensor.data_ptr()}")

# Check for shared memory
is_reshape_copy_shared = non_contiguous_tensor.data_ptr() == reshape_copy_tensor.data_ptr()
print(f"Memory Shared (reshape/copy)? {is_reshape_copy_shared} (It's a COPY)")

Original Tensor: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Original Data Pointer: 756042048
---
## Case A: view() (Always creates a View if Contiguous) ##
View Tensor: tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])
View Data Pointer: 756042048
Memory Shared (view)? True (It's a VIEW)
--------------------
## Case B: reshape() (Creates a View when Contiguous) ##
Reshape (View) Tensor: tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]])
Reshape (View) Data Pointer: 756042048
Memory Shared (reshape/view)? True (It's a VIEW)
--------------------
## Case C: reshape() (Creates a Copy when Non-Contiguous) ##
non_contiguous_tensor(view): tensor([[0, 5],
        [1, 6],
        [2, 7],
        [3, 8],
        [4, 9]])
Non-Contiguous Data Pointer: 668047104
Reshape (Copy) Tensor: tensor([0, 5, 1, 6, 2, 7, 3, 8, 4, 9])
Reshape (Copy) Data Pointer: 767621952
Memory Shared (reshape/copy)? False (It's a COPY)


In [2]:
print("\n========== STEP 1: CREATE CONTIGUOUS TENSOR x ==========\n")

# Create tensor x
x = torch.tensor([[1, 2, 3],
                  [4, 5, 6]])

# Metadata of x
print("Tensor x metadata:")
print("shape  :", x.shape)
print("stride :", x.stride())
print("is_contiguous:", x.is_contiguous())

# Memory view of x
print("\nMemory storage of x:")
print(list(x.storage()))

# Print x
print("\nTensor x:")
print(x)


print("\n========== STEP 2: TRANSPOSE → CREATE y ==========\n")

# Transpose
y = x.t()

# Orientation change
print("Tensor y = x.t() (transpose):")
print(y)

print("\nNotice how orientation changes (rows ↔ columns)")
print("Elements appear shuffled visually, but memory is unchanged.")

# Metadata of y
print("\nTensor y metadata:")
print("shape  :", y.shape)
print("stride :", y.stride())
print("is_contiguous:", y.is_contiguous())

# Memory view of y
print("\nMemory storage of y (same as x):")
print(list(y.storage()))


print("\n========== STEP 3: ROW-MAJOR LOGICAL ACCESS OF y ==========\n")

print("Row-major traversal of y (logical order):")

logical_order = []
for i in range(y.shape[0]):
    for j in range(y.shape[1]):
        val = y[i, j].item()
        logical_order.append(val)
        print(f"y[{i},{j}] = {val}")

print("\nLogical row-major order of y:")
print(logical_order)


print("\n========== STEP 4: MEMORY ORDER vs LOGICAL ORDER ==========\n")

memory_order = list(y.storage())

print("Memory order (actual storage):")
print(memory_order)

print("\nComparison:")
print("Logical row-major order :", logical_order)
print("Actual memory order     :", memory_order)


print("\n========== STEP 5: CONTIGUITY CHECK ==========\n")

if not y.is_contiguous():
    print("❌ y is NOT contiguous")
    print("Reason: logical row-major order ≠ memory order")
else:
    print("✅ y is contiguous")


========== STEP 1: CREATE CONTIGUOUS TENSOR x ==========

Tensor x metadata:
shape  : torch.Size([2, 3])
stride : (3, 1)
is_contiguous: True

Memory storage of x:
[1, 2, 3, 4, 5, 6]

Tensor x:
tensor([[1, 2, 3],
        [4, 5, 6]])

========== STEP 2: TRANSPOSE → CREATE y ==========

Tensor y = x.t() (transpose):
tensor([[1, 4],
        [2, 5],
        [3, 6]])

Notice how orientation changes (rows ↔ columns)
Elements appear shuffled visually, but memory is unchanged.

Tensor y metadata:
shape  : torch.Size([3, 2])
stride : (1, 3)
is_contiguous: False

Memory storage of y (same as x):
[1, 2, 3, 4, 5, 6]

========== STEP 3: ROW-MAJOR LOGICAL ACCESS OF y ==========

Row-major traversal of y (logical order):
y[0,0] = 1
y[0,1] = 4
y[1,0] = 2
y[1,1] = 5
y[2,0] = 3
y[2,1] = 6

Logical row-major order of y:
[1, 4, 2, 5, 3, 6]

========== STEP 4: MEMORY ORDER vs LOGICAL ORDER ==========

Memory order (actual storage):
[1, 2, 3, 4, 5, 6]

Comparison:
Logical row-major order : [1, 4, 2, 5, 3, 6

/tmp/ipython-input-1866011434.py:15: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  print(list(x.storage()))
